In [1]:
import pandas as pd
import utils
import os
import ft_ensemble
import numpy as np
import ee

In [2]:
# Path to the pretrained models folder
path_2_9km_models =   'pretrained_models/DenseSM_9km'

In [ ]:
# path to training data CSV file
input_fine = pd.read_csv('/mnt/data2tb/Transfer-DenseSM-E_pack/fusion/fusion_full.csv',index_col='s_index')
print(len(input_fine))
# Drop all rows where 'sm' (soil moisture) values are greater than 0.7 
input_fine = input_fine[input_fine['sm'] <= 0.7]
input_fine = input_fine[input_fine['sm'] > 0.0]
print(len(input_fine))

162163
159336


In [4]:
# Print names of sub dataset used for training
np.unique(input_fine['network'])

array(['CHINA_100m', 'CHINA_1km', 'INDIA_100m', 'INDIA_1km', 'VN'],
      dtype=object)

In [5]:
setup = {'lr':4e-4, # learning rate
        'epoch_Num':500,# number of epochs
        'swa_start':40,
        'alpha':0.7,# alpha in eq. 3
        'beta':'auto',# beta in eq. 3 and was determined by eq.4
        'domain_type':'coral',
        'mv_type':'MAPE',
        'ex':'ft12_models',
        'batchS':1024,# size of batch
        'br':1}# control the numbmer of unlabled samples and 9km samples, 1 means 1*batch_size
network_name=['CHINA_100m', 'CHINA_1km', 'INDIA_100m', 'INDIA_1km', 'VN']
base_dir = 'Demo' #%network_name
model_dir_e = os.path.join(base_dir,setup['ex'])
# if not os.path.exists(model_dir_e):
#     os.mkdir(model_dir_e)
os.makedirs(model_dir_e, exist_ok=True)

In [6]:
if isinstance(setup['beta'], str):
    print(setup['beta'])

auto


### Train with specific split ratio
Chia bộ dữ liệu theo tỉ lệ train-val là 90/10. Bộ test sẽ sử dụng bộ dữ liệu Thái Bình

In [ ]:
from sklearn.model_selection import train_test_split

# name folder to save the models
nsample = 'deletethis'
model_dir = os.path.join(model_dir_e, '%s'%nsample)
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

# 3 is the number of repeats (we want 3 different set of models)
for r in range(3):
    if isinstance(setup['beta'], str):
        model_dir_r = os.path.join(model_dir, 'a%s_b%s_r%s'%(int(setup['alpha']*100), setup['beta'], str(r)))
    else:
        model_dir_r = os.path.join(model_dir,'a%s_b%s_r%s'%(int(setup['alpha']*100),int(setup['beta']*100),str(r)))
    if not os.path.exists(model_dir_r):
        os.mkdir(model_dir_r)
    s_index = input_fine.index.to_list()
    
    # Split the dataset into train and val set (9:1)
    train_index, test_index = train_test_split(s_index, test_size=0.1, random_state=10, shuffle=True)

    # Print the number of samples in train and test sets
    print(len(train_index))
    print(len(test_index))

    # Prepare the training and validation data
    train_val_index = [train_index, test_index]
    trainloader, val_data, targetloader, train_data = utils.prepare_train_val_data_highres(input_fine, train_val_index, setup['batchS'], setup['br'])
    
    # Build the DenseSM models from the pretrained models
    modelX = ft_ensemble.Build_DenseSM(path_2_9km_models)
    modelX.rebuild_DenseSME()
    data = {'sl': trainloader,
            'tl':targetloader,
            'cl':None,
            'val_data': val_data,
            'train_data':train_data}
    
    # Prepare the setup for finetuning
    print(f'Finetune sample: {nsample} repeat: {r}')
    ft = ft_ensemble.FinetuneModel(setup, data)
    
    # Finetuning the model
    ft.ft_ensemble(modelX, model_dir_r)
    df, res, m_specific_y=ft_ensemble.ensemble_results(model_dir_r, val_data)
    res.scatter_density_fig(os.path.join(model_dir_r,'ensemble.jpg'))
    print(res.stat_3d)

143402
15934
Finetune sample: deletethis repeat: 0
[-0.008  0.945  0.041  0.04   0.991]
143402
15934
Finetune sample: deletethis repeat: 1


In [ ]:
# input_9km=utils.get_9km_of_a_network(input_fine,input_9km,network_name)
# for nsample in size_r:
#     model_dir = os.path.join(model_dir_e,'n_%s'%str(nsample))
#     if not os.path.exists(model_dir):
#         os.mkdir(model_dir)

#     for r in range(10):# 10 implementation
#         if isinstance(setup['beta'],str):
#             model_dir_r = os.path.join(model_dir,'a%s_b%s_r%s'%(int(setup['alpha']*100),setup['beta'],str(r)))
#         else:
#             model_dir_r = os.path.join(model_dir,'a%s_b%s_r%s'%(int(setup['alpha']*100),int(setup['beta']*100),str(r)))
#         if not os.path.exists(model_dir_r):
#             os.mkdir(model_dir_r)
#         train_val_index = utils.get_train_val_index(input_fine,nsample,network=network_name)
#         trainloader, val_data,targetloader,train_data,loader_9km = utils.prepare_train_val_data_multiscale(input_fine,train_val_index,setup['batchS'],setup['br'])
#         modelX=ft_ensemble.Build_DenseSM(path_2_9km_models)
#         modelX.rebuild_DenseSME()
#         data={'sl':trainloader,
#              'tl':targetloader,
#              'cl':loader_9km,
#              'val_data':val_data,
#              'train_data':train_data}
#         print(f'Finetune sample:{nsample} repeat:{r}')
#         ft=ft_ensemble.FinetuneModel(setup,data)
#         ft.ft_ensemble(modelX,model_dir_r)
#         df, res, m_specific_y=ft_ensemble.ensemble_results(model_dir_r, val_data)
#         res.scatter_density_fig(os.path.join(model_dir_r,'ensemble.jpg'))
#         print(res.stat_3d)